# Integrantes

* Fabrizzio Mura
* Joaquin Tapia
* Matías Bugueño

## Librerias necesarias

In [1]:
from read_file import read_file
from TTP import GA, objective_function
from TTP.crossovers import SegmentSimple
from TTP.mutations import ResettingScramble, ResettingSwamp, FlipScramble, FlipSwamp
from TTP.Greedy import greedy_ttp
from TTP.common import Item
from read_file import read_file
import numpy as np

## Funciones Necesarias

In [2]:
def get_GA_items(thief_items: list, items: list[Item]) -> np.ndarray:
    items_size = len(items)
    thief_items_size = len(thief_items)
    GA_items = np.zeros((items_size), dtype= int)

    for i in range(items_size):
        for j in range(thief_items_size):
            if items[i].get_house_id() == thief_items[j]:
                GA_items[i] = 1

    return GA_items

## Preparación de data

In [3]:
name, thief , cities , items, ratio = read_file("./data/a280-ttp/a280_n279_bounded-strongly-corr_01.ttp")

data = {'Houses': cities,
        'Items': items,
        'Thief': thief,
        'Ratio': ratio}

## Algoritmo Greedy

In [4]:
route, thief_items, total_time, profit, execution_time = greedy_ttp(thief, cities)
thief_items = [item.get_house_id() for item in thief_items]
thief_items = get_GA_items(thief_items, items)
sol = (route, thief_items)

print(f"Tiempo de ejecución: {round(execution_time, 3)}[s]")
print(f"Tiempo ladrón: {round(total_time, 3)}[s]")
print(f"Dinero: ${profit}")
print(f"Función objetivo: {round(objective_function(data, sol), 3)}")
print(f'Ruta del ladron: {route}')

Tiempo de ejecución: 0.101[s]
Tiempo ladrón: 3606.436[s]
Dinero: $26586
Función objetivo: -41658.797
Ruta del ladron: [280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 1, 242, 243, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 244, 247, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 210, 207, 208, 209, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 138, 137, 136, 135, 134, 270, 269, 268, 267, 140, 139, 148, 141, 142, 143, 144, 145, 146, 147, 149, 150, 178, 151, 152, 156, 153, 155, 154, 129, 130, 131, 132, 133, 15, 249, 248, 206, 205, 204, 203, 202, 200, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 181, 176, 177, 180, 179, 160, 159, 158, 157, 119, 120, 121, 122, 123, 124, 34, 33, 36, 35, 38, 37, 39, 40, 41, 42, 43, 60, 61, 118, 117, 

## Algoritmo Genético

In [ ]:
route, thief_items = GA(data, mutate_ratio= 0.5, truncation_ratio= 0.2, epochs= 50, 
                    population_size= 200, total_cities= len(cities), 
                    total_objects= len(items), mutation= ResettingScramble(), crossover= SegmentSimple())